In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datetime import datetime

### NLP PART

In [2]:
cd drive/My Drive/Project ML

/content/drive/My Drive/Project ML


In [0]:
clear_data = pd.read_excel('Clear1.xlsx')
covid_data = pd.read_excel('Covid.xlsx')

In [0]:
clear_data_f = clear_data[clear_data.event_type == 0]
clear_data_f = clear_data_f.drop('Unnamed: 0', axis = 1)

covid_data_f = covid_data[covid_data.event_type == 1]
covid_data_f = covid_data_f.drop('Unnamed: 0', axis = 1)

In [0]:
united_data = clear_data_f.append(covid_data_f).sample(frac=1).reset_index(drop=True)

In [6]:
united_data.head()


,event_type,description
0,0,tv station refuses adoption show a tv station...
1,0,casino royale is next bond movie casino royal...
2,0,web radio takes spanish rap global spin the r...
3,0,tenyear tragedy of missing manic richey edwar...
4,0,holmes back on form in birmingham double olym...


In [7]:
united_data['event_type'].unique()

array([0, 1])

In [8]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

     |████████████████████████████████| 1.0MB 1.4MB/s 
     |████████████████████████████████| 870kB 37.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=4229ae05249e1d4e4e877dff5ef747791be4c2393093f3729bca8068763d201b
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [9]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 1.4MB/s 


In [10]:
!pip install transformers 
!pip install pytorch-transformers

     |████████████████████████████████| 552kB 1.4MB/s 
     |████████████████████████████████| 3.7MB 6.5MB/s 
     |████████████████████████████████| 184kB 1.4MB/s 


In [11]:
import torch
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading BERT tokenizer...


In [0]:
def make_appropriate_size(A):
  '''
  Tokenazier can handle only with text of size not bigger than 512 symbols. In order to satisfy this condition
  we want to take first 512 each test, split it and threw last element to ensure that there will not be cutten words.
   A - list of elements
  '''
  A = A[:512]
  A = A.split(sep = ' ')
  s = ' '
  A = s.join(A[:-1])
  return A

In [0]:
united_data['desc_tr'] = united_data['description'].apply(lambda x: make_appropriate_size(x))
united_data_f = united_data.drop('description', axis = 1)

In [14]:
united_data_f.head()

,event_type,desc_tr
0,0,tv station refuses adoption show a tv station...
1,0,casino royale is next bond movie casino royal...
2,0,web radio takes spanish rap global spin the r...
3,0,tenyear tragedy of missing manic richey edwar...
4,0,holmes back on form in birmingham double olym...


In [0]:
sentences = united_data_f.desc_tr.values
labels = united_data_f.event_type.values

In [16]:
max_len = 0

for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  186


In [17]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 170,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  tv station refuses adoption show  a tv station in the us has refused to show a controversial new series where adopted children try and pick their birth father  and win a cash prize  the wraztv fox affiliate in north carolina was the only one of  stations to refuse mondays show whos your daddy promises   to the contestant if she correctly identifies her father it was met with protests by the national council for adoption which said it exploits sensitive emotions it exploits the sensitive emotions of
Token IDs: tensor([  101,  2694,  2276, 10220,  9886,  2265,  1037,  2694,  2276,  1999,
         1996,  2149,  2038,  4188,  2000,  2265,  1037,  6801,  2047,  2186,
         2073,  4233,  2336,  3046,  1998,  4060,  2037,  4182,  2269,  1998,
         2663,  1037,  5356,  3396,  1996, 23277, 10936,  9189,  4419,  8727,
         1999,  2167,  3792,  2001,  1996,  2069,  2028,  1997,  3703,  2000,
        10214, 28401,  2265,  2040,  2015,  2115,  8600, 10659,  2000,  1996,
      

In [18]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 80-10 train-validation split.

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

3,180 training samples
  796 validation samples


In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

# Create the DataLoaders training and validation sets.

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [0]:
device = torch.device('cpu')

from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for our case of binary classification.
 
    output_attentions = False, 
    output_hidden_states = False, 
)

model.to(device)


In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [0]:
from transformers import get_linear_schedule_with_warmup

epochs = 4 #as BERT reccomend

total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [139]:

import random
import numpy as np

# Set the seed value all over the place to make this reproducible.
seed_val = 2

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Time of training the  epoch.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        #clear previously calculated gradients
        model.zero_grad()        

        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        

        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    199.    Elapsed: 0:09:58.
  Batch    80  of    199.    Elapsed: 0:19:40.
  Batch   120  of    199.    Elapsed: 0:29:17.
  Batch   160  of    199.    Elapsed: 0:38:49.

  Average training loss: 0.08
  Training epcoh took: 0:47:57

Running Validation...
  Accuracy: 1.00
  Validation Loss: 0.02
  Validation took: 0:03:13

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    199.    Elapsed: 0:09:14.
  Batch    80  of    199.    Elapsed: 0:18:29.
  Batch   120  of    199.    Elapsed: 0:27:43.
  Batch   160  of    199.    Elapsed: 0:37:07.

  Average training loss: 0.01
  Training epcoh took: 0:46:18

Running Validation...
  Accuracy: 1.00
  Validation Loss: 0.00
  Validation took: 0:03:22

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    199.    Elapsed: 0:09:34.
  Batch    80  of    199.    Elapsed: 0:19:34.
  Batch   120  of    199.    Elapsed: 0:30:23.
  Batch   160  of    199.    Elapsed: 0:41:40.



In [141]:
import os

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to D:/model_save/


('D:/model_save/vocab.txt',
 'D:/model_save/special_tokens_map.json',
 'D:/model_save/added_tokens.json')

In [143]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
